<a href="https://colab.research.google.com/github/SOWMYASRI7/DL-ASSIGNMENT-2/blob/main/DL_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, SimpleRNN, LSTM, GRU, Dense
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
import unicodedata
import re

In [ ]:
import tensorflow as tf

# Load data (with Devanagari → Latin → freq columns)
def load_data(file_path, num_samples=None):
    input_texts = []
    target_texts = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if num_samples and i >= num_samples:
                break
            parts = line.strip().split('\t')
            if len(parts) >= 2:
                latin_input = preprocess_sentence(parts[1])  # Apply preprocessing
                devanagari_output = preprocess_sentence(parts[0])  # Apply preprocessing
                input_texts.append(latin_input)
                target_texts.append(devanagari_output)
    return input_texts, target_texts


# Tokenize character-wise
def tokenize_char(sequences):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, filters='')
    tokenizer.fit_on_texts(sequences)
    sequences_tensor = tokenizer.texts_to_sequences(sequences)
    return sequences_tensor, tokenizer

# Pad sequences
def pad_sequences(sequences, maxlen=None):
    return tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=maxlen, padding='post')

# Full pipeline
def preprocess_dataset(file_path, num_samples=None):
    input_texts, target_texts = load_data(file_path, num_samples)

    input_tensor_raw, inp_tokenizer = tokenize_char(input_texts)
    target_tensor_raw, targ_tokenizer = tokenize_char(target_texts)

    max_input_len = max(len(seq) for seq in input_tensor_raw)
    max_target_len = max(len(seq) for seq in target_tensor_raw)

    input_tensor = pad_sequences(input_tensor_raw, max_input_len)
    target_tensor = pad_sequences(target_tensor_raw, max_target_len)

    return (input_tensor, target_tensor,
            inp_tokenizer, targ_tokenizer,
            max_input_len, max_target_len)

In [ ]:
# Cell 4: Display Sample of Preprocessed Data
num_samples_to_view = 5

print("Sample of Preprocessed Data:")
for i in range(num_samples_to_view):
    decoded_input = ''.join(inp_tokenizer.index_word.get(idx, '') for idx in input_tensor[i] if idx != 0)
    decoded_target = ''.join(targ_tokenizer.index_word.get(idx, '') for idx in target_tensor[i] if idx != 0)
    print(f"Latin Input (Encoded): {input_tensor[i]}")
    print(f"Latin Input (Decoded): {decoded_input}")
    print(f"Devanagari Target (Encoded): {target_tensor[i]}")
    print(f"Devanagari Target (Decoded): {decoded_target}")
    print('-' * 50)



Sample of Preprocessed Data:
Latin Input (Encoded): [1 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Latin Input (Decoded): an
Devanagari Target (Encoded): [ 1 10  8  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
Devanagari Target (Decoded): 	अं

--------------------------------------------------
Latin Input (Encoded): [ 1  3  4 19  1  3  2  7  0  0  0  0  0  0  0  0  0  0]
Latin Input (Decoded): ankganit
Devanagari Target (Encoded): [ 1 10  8  4 28 40  9 11  2  0  0  0  0  0  0  0  0  0  0  0]
Devanagari Target (Decoded): 	अंकगणित

--------------------------------------------------
Latin Input (Encoded): [11  3 16 13  8  0  0  0  0  0  0  0  0  0  0  0  0  0]
Latin Input (Decoded): uncle
Devanagari Target (Encoded): [ 1 10  8  4 13  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
Devanagari Target (Decoded): 	अंकल

--------------------------------------------------
Latin Input (Encoded): [ 1  3  4 11  6  0  0  0  0  0  0  0  0  0  0  0  0  0]
Latin Input (Decoded): ankur
Devanagari Target (Enc

In [ ]:
from sklearn.model_selection import train_test_split

input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(
    input_tensor, target_tensor, test_size=0.2, random_state=42
)
print(f"\nTraining input shape: {input_tensor_train.shape}")
print(f"Validation input shape: {input_tensor_val.shape}")
print(f"Training target shape: {target_tensor_train.shape}")
print(f"Validation target shape: {target_tensor_val.shape}")



Training input shape: (8000, 18)
Validation input shape: (2000, 18)
Training target shape: (8000, 20)
Validation target shape: (2000, 20)


In [ ]:
embedding_dim = 64
hidden_units = 128
cell_type = 'lstm'  # You can experiment with 'rnn' or 'gru'
num_encoder_layers = 1

encoder_inputs = tf.keras.layers.Input(shape=(None,), name='encoder_input')
encoder_embedding = tf.keras.layers.Embedding(len(inp_tokenizer.word_index) + 1, embedding_dim, name='encoder_embedding')(encoder_inputs)
encoder_outputs = encoder_embedding
encoder_states = []

for i in range(num_encoder_layers):
    if cell_type == 'lstm':
        encoder_lstm = tf.keras.layers.LSTM(hidden_units, return_sequences=True, return_state=True, name=f'encoder_lstm_{i}')
        encoder_outputs, state_h, state_c = encoder_lstm(encoder_outputs)
        encoder_states.extend([state_h, state_c])
    elif cell_type == 'gru':
        encoder_gru = tf.keras.layers.GRU(hidden_units, return_sequences=True, return_state=True, name=f'encoder_gru_{i}')
        encoder_outputs, state_h = encoder_gru(encoder_outputs)
        encoder_states.append(state_h)
    elif cell_type == 'rnn':
        encoder_rnn = tf.keras.layers.SimpleRNN(hidden_units, return_sequences=True, return_state=True, name=f'encoder_rnn_{i}')
        encoder_outputs, state_h = encoder_rnn(encoder_outputs)
        encoder_states.append(state_h)

encoder = tf.keras.models.Model(encoder_inputs, encoder_states)


In [ ]:
# Cell 7: Define the Decoder Model
num_decoder_layers = 1

decoder_inputs = tf.keras.layers.Input(shape=(None,), name='decoder_input')
decoder_embedding = tf.keras.layers.Embedding(len(targ_tokenizer.word_index) + 1, embedding_dim, name='decoder_embedding')(decoder_inputs)
decoder_outputs = decoder_embedding
decoder_states_inputs = []
decoder_states = []

for i in range(num_decoder_layers):
    if cell_type == 'lstm':
        state_h_in = tf.keras.layers.Input(shape=(hidden_units,), name=f'decoder_state_h_in_{i}')
        state_c_in = tf.keras.layers.Input(shape=(hidden_units,), name=f'decoder_state_c_in_{i}')
        decoder_states_inputs.extend([state_h_in, state_c_in])
        decoder_lstm = tf.keras.layers.LSTM(hidden_units, return_sequences=True, return_state=True, name=f'decoder_lstm_{i}')
        decoder_outputs, state_h_out, state_c_out = decoder_lstm(decoder_outputs, initial_state=[state_h_in, state_c_in])
        decoder_states.extend([state_h_out, state_c_out])
    elif cell_type == 'gru':
        state_h_in = tf.keras.layers.Input(shape=(hidden_units,), name=f'decoder_state_h_in_{i}')
        decoder_states_inputs.append(state_h_in)
        decoder_gru = tf.keras.layers.GRU(hidden_units, return_sequences=True, return_state=True, name=f'decoder_gru_{i}')
        decoder_outputs, state_h_out = decoder_gru(decoder_outputs, initial_state=state_h_in)
        decoder_states.append(state_h_out)
    elif cell_type == 'rnn':
        state_h_in = tf.keras.layers.Input(shape=(hidden_units,), name=f'decoder_state_h_in_{i}')
        decoder_states_inputs.append(state_h_in)
        decoder_rnn = tf.keras.layers.SimpleRNN(hidden_units, return_sequences=True, return_state=True, name=f'decoder_rnn_{i}')
        decoder_outputs, state_h_out = decoder_rnn(decoder_outputs, initial_state=state_h_in)
        decoder_states.append(state_h_out)

decoder_dense = tf.keras.layers.Dense(len(targ_tokenizer.word_index) + 1, activation='softmax', name='decoder_output')
decoder_outputs = decoder_dense(decoder_outputs)
decoder = tf.keras.models.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)


In [ ]:
# Cell 8: Define the Seq2Seq Model for Training
model_inputs = [encoder_inputs, decoder_inputs]
initial_decoder_states = encoder_states[:num_decoder_layers] if cell_type != 'lstm' else encoder_states[:2*num_decoder_layers]

# Ensure initial_decoder_states is a list of tensors
if not isinstance(initial_decoder_states, list):
    initial_decoder_states = [initial_decoder_states]

model_outputs = decoder([decoder_inputs] + initial_decoder_states)[0]
model = tf.keras.models.Model(model_inputs, model_outputs)

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss_fn)

# Print model summary (optional)
model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_input       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_embedding   │ (None, None, 64)  │      1,728 │ encoder_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_input       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_lstm_0      │ [(None, None,     │     98,816 │ encoder_embeddin… │
│ (LSTM)              │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_5        │ [(None, None,     │    111,168 │ decoder_input[0]… │
│ (Functional)        │ 64), (None, 128), │            │ encoder_lstm_0[0… │
│                     │ (None, 128)]      │            │ encoder_lstm_0[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 211,712 (827.00 KB)

 Trainable params: 211,712 (827.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import unicodedata
import re
from tensorflow.keras.callbacks import ModelCheckpoint

# --- Define Preprocessing Functions ---
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFKD', s) if unicodedata.category(c) != 'Mn')

def preprocess_latin(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'["\\]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w = w.strip()
    w = '\t' + w + '\n'
    return w

def preprocess_devanagari(w):
    w = w.strip()
    w = '\t' + w + '\n'
    return w

def load_data(file_path, num_samples=None):
    input_texts = []
    target_texts = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if num_samples and i >= num_samples:
                break
            parts = line.strip().split('\t')
            if len(parts) >= 2:
                latin_input = preprocess_latin(parts[1])
                devanagari_output = preprocess_devanagari(parts[0])
                input_texts.append(latin_input)
                target_texts.append(devanagari_output)
    return input_texts, target_texts

def tokenize_char(sequences):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, filters='')
    tokenizer.fit_on_texts(sequences)
    sequences_tensor = tokenizer.texts_to_sequences(sequences)
    return sequences_tensor, tokenizer

def pad_sequences(sequences, maxlen=None, padding='post'):
    return tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=maxlen, padding=padding)

def preprocess_dataset(file_path, num_samples=None):
    input_texts, target_texts = load_data(file_path, num_samples)
    input_tensor_raw, inp_tokenizer = tokenize_char(input_texts)
    target_tensor_raw, targ_tokenizer = tokenize_char(target_texts)
    max_input_len = max(len(seq) for seq in input_tensor_raw)
    max_target_len = max(len(seq) for seq in target_tensor_raw)
    input_tensor = pad_sequences(input_tensor_raw, max_input_len)
    target_tensor = pad_sequences(target_tensor_raw, max_target_len)
    return input_tensor, target_tensor, inp_tokenizer, targ_tokenizer, max_input_len, max_target_len

# --- Build Seq2Seq Model ---
def build_model(num_encoder_tokens, num_decoder_tokens, embedding_dim=256, latent_dim=256):
    encoder_inputs = Input(shape=(None,), name='encoder_inputs')
    encoder_embedding = Embedding(num_encoder_tokens, embedding_dim, name='encoder_embedding')(encoder_inputs)
    encoder_lstm, state_h, state_c = LSTM(latent_dim, return_state=True, name='encoder_lstm')(encoder_embedding)
    encoder_states = [state_h, state_c]

    decoder_inputs = Input(shape=(None,), name='decoder_inputs')
    decoder_embedding = Embedding(num_decoder_tokens, embedding_dim, name='decoder_embedding')(decoder_inputs)
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm')
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
    decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
    decoder_outputs = decoder_dense(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    return model

# --- Inference Models ---
def build_inference_models(model, latent_dim, num_encoder_tokens, num_decoder_tokens):
    encoder_inputs = model.input[0]
    encoder_embedding = model.get_layer('encoder_embedding')(encoder_inputs)
    encoder_outputs, state_h_enc, state_c_enc = model.get_layer('encoder_lstm')(encoder_embedding)
    encoder_states = [state_h_enc, state_c_enc]
    encoder_model = Model(encoder_inputs, encoder_states)

    decoder_inputs = model.input[1]
    decoder_state_input_h = Input(shape=(latent_dim,), name='input_h')
    decoder_state_input_c = Input(shape=(latent_dim,), name='input_c')
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_embedding = model.get_layer('decoder_embedding')(decoder_inputs)
    decoder_lstm = model.get_layer('decoder_lstm')
    decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
    decoder_states = [state_h_dec, state_c_dec]
    decoder_dense = model.get_layer('decoder_dense')
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

    return encoder_model, decoder_model

# --- Decode Sequence ---
def decode_sequence(input_seq, encoder_model, decoder_model, targ_tokenizer, max_target_len):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = targ_tokenizer.word_index['\t']
    stop_condition = False
    decoded_sentence = ''

    reverse_target_char_index = dict((i, char) for char, i in targ_tokenizer.word_index.items())

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index.get(sampled_token_index, '')
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or len(decoded_sentence) > max_target_len):
            stop_condition = True

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence

# --- Train the Model ---
def train_model(file_path, num_samples=10000, embedding_dim=256, latent_dim=256, batch_size=64, epochs=30):
    input_tensor, target_tensor, inp_tokenizer, targ_tokenizer, max_input_len, max_target_len = preprocess_dataset(file_path, num_samples)

    # Split only once
    input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

    num_encoder_tokens = len(inp_tokenizer.word_index) + 1
    num_decoder_tokens = len(targ_tokenizer.word_index) + 1

    # Create decoder input/target for training and validation
    decoder_input_train = target_tensor_train[:, :-1]
    decoder_target_train = target_tensor_train[:, 1:]
    decoder_input_val = target_tensor_val[:, :-1]
    decoder_target_val = target_tensor_val[:, 1:]

    model = build_model(num_encoder_tokens, num_decoder_tokens, embedding_dim, latent_dim)
    model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

    model.summary()

    checkpoint_callback = ModelCheckpoint('model_weights.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)

    model.fit([input_tensor_train, decoder_input_train], decoder_target_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=([input_tensor_val, decoder_input_val], decoder_target_val),
              callbacks=[checkpoint_callback],
              verbose=1)

    encoder_model, decoder_model = build_inference_models(model, latent_dim, num_encoder_tokens, num_decoder_tokens)
    return model, encoder_model, decoder_model, inp_tokenizer, targ_tokenizer, max_target_len


    # Define checkpoint callback to save the best model
    checkpoint_callback = ModelCheckpoint('model_weights.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)

    model.fit([input_tensor_train, decoder_input_data], decoder_target_data,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=([input_tensor_val, decoder_input_data], decoder_target_data),
              callbacks=[checkpoint_callback],
              verbose=1)  # Adds more training feedback

    encoder_model, decoder_model = build_inference_models(model, latent_dim, num_encoder_tokens, num_decoder_tokens)
    return model, encoder_model, decoder_model, inp_tokenizer, targ_tokenizer, max_target_len


In [4]:
model, encoder_model, decoder_model, inp_tokenizer, targ_tokenizer, max_target_len = train_model('/content/hi.translit.sampled.train.tsv', num_samples=10000)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_embedding   │ (None, None, 256) │      7,424 │ encoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_embedding   │ (None, None, 256) │     16,384 │ decoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_lstm (LSTM) │ [(None, 256),     │    525,312 │ encoder_embeddin… │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, None,     │    525,312 │ decoder_embeddin… │
│                     │ 256), (None,      │            │ encoder_lstm[0][… │
│                     │ 256), (None,      │            │ encoder_lstm[0][… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_dense       │ (None, None, 64)  │     16,448 │ decoder_lstm[0][… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,090,880 (4.16 MB)

 Trainable params: 1,090,880 (4.16 MB)

 Non-trainable params: 0 (0.00 B)

...

125/125 ━━━━━━━━━━━━━━━━━━━━ 40s 290ms/step - loss: 0.4369 - val_loss: 0.4926
Epoch 27/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step - loss: 0.4113
Epoch 27: val_loss improved from 0.49260 to 0.47062, saving model to model_weights.h5


125/125 ━━━━━━━━━━━━━━━━━━━━ 38s 301ms/step - loss: 0.4113 - val_loss: 0.4706
Epoch 28/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step - loss: 0.3842
Epoch 28: val_loss improved from 0.47062 to 0.45031, saving model to model_weights.h5


125/125 ━━━━━━━━━━━━━━━━━━━━ 40s 292ms/step - loss: 0.3843 - val_loss: 0.4503
Epoch 29/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step - loss: 0.3673
Epoch 29: val_loss improved from 0.45031 to 0.43046, saving model to model_weights.h5


125/125 ━━━━━━━━━━━━━━━━━━━━ 46s 329ms/step - loss: 0.3673 - val_loss: 0.4305
Epoch 30/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step - loss: 0.3401
Epoch 30: val_loss improved from 0.43046 to 0.42762, saving model to model_weights.h5


125/125 ━━━━━━━━━━━━━━━━━━━━ 77s 293ms/step - loss: 0.3401 - val_loss: 0.4276


In [ ]:
if __name__ == '__main__':
    # File paths
    train_file = '/content/hi.translit.sampled.train.tsv'
    test_file = '/content/hi.translit.sampled.test.tsv'
    num_samples = 10000

    # Train the model on training data
    (model, encoder_model, decoder_model,
     inp_tokenizer, targ_tokenizer,
     max_target_len, train_input_texts, train_target_texts) = \
        train_model(train_file, num_samples)

    # Preprocess test data (use all samples)
    (test_input_tensor, test_target_tensor,
     _, _, _, _,
     test_input_texts, test_target_texts) = \
        preprocess_dataset(test_file, None)

    # --- Sample Predictions from Test Set ---
    print("Sample predictions on test set:")
    for i in range(5):
        seq = test_input_tensor[i:i+1]
        latin = test_input_texts[i].strip()
        true_dev = test_target_texts[i].strip()
        pred_dev = decode_sequence(seq, encoder_model, decoder_model, targ_tokenizer, max_target_len).strip()
        print(f"{i+1}. Latin: {latin}  | True: {true_dev}  | Pred: {pred_dev}")

    # --- Compute Test Accuracy ---
    num_test = len(test_input_texts)
    correct = 0
    for i in range(num_test):
        seq = test_input_tensor[i:i+1]
        true_txt = test_target_texts[i].strip()
        pred_txt = decode_sequence(seq, encoder_model, decoder_model, targ_tokenizer, max_target_len).strip()
        if pred_txt == true_txt:
            correct += 1
    accuracy = correct / num_test if num_test > 0 else 0
    print(f'Test set accuracy over {num_test} samples: {accuracy:.2%}')


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_embedding   │ (None, None, 256) │      7,424 │ encoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_embedding   │ (None, None, 256) │     16,384 │ decoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_lstm (LSTM) │ [(None, 256),     │    525,312 │ encoder_embeddin… │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, None,     │    525,312 │ decoder_embeddin… │
│                     │ 256), (None,      │            │ encoder_lstm[0][… │
│                     │ 256), (None,      │            │ encoder_lstm[0][… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_dense       │ (None, None, 64)  │     16,448 │ decoder_lstm[0][… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,090,880 (4.16 MB)

 Trainable params: 1,090,880 (4.16 MB)

 Non-trainable params: 0 (0.00 B)

...

125/125 ━━━━━━━━━━━━━━━━━━━━ 37s 295ms/step - loss: 0.8431 - val_loss: 0.8247
Epoch 9/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step - loss: 0.8201
Epoch 9: val_loss improved from 0.82468 to 0.82388, saving model to model_weights.h5


125/125 ━━━━━━━━━━━━━━━━━━━━ 36s 288ms/step - loss: 0.8201 - val_loss: 0.8239
Epoch 10/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step - loss: 0.7925
Epoch 10: val_loss improved from 0.82388 to 0.79733, saving model to model_weights.h5


125/125 ━━━━━━━━━━━━━━━━━━━━ 44s 312ms/step - loss: 0.7925 - val_loss: 0.7973
Epoch 11/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step - loss: 0.7775
Epoch 11: val_loss improved from 0.79733 to 0.77405, saving model to model_weights.h5


125/125 ━━━━━━━━━━━━━━━━━━━━ 40s 308ms/step - loss: 0.7775 - val_loss: 0.7741
Epoch 12/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step - loss: 0.7464
Epoch 12: val_loss improved from 0.77405 to 0.74198, saving model to model_weights.h5


125/125 ━━━━━━━━━━━━━━━━━━━━ 40s 299ms/step - loss: 0.7464 - val_loss: 0.7420
Epoch 13/30
 70/125 ━━━━━━━━━━━━━━━━━━━━ 15s 284ms/step - loss: 0.7208

In [ ]:
# Preprocess the dataset
file_path = '/content/hi.translit.sampled.train.tsv'  # Update with the correct file path
input_tensor, target_tensor, inp_tokenizer, targ_tokenizer, max_input_len, max_target_len = preprocess_dataset(file_path, num_samples=10000)

# Example: Check the preprocessed data
print("Sample Latin Input (Decoded):", input_tensor[0])
print("Sample Devanagari Target (Decoded):", target_tensor[0])

# Now, you can use the preprocessed data for training your Seq2Seq model.


Sample Latin Input (Decoded): [2 1 5 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Sample Devanagari Target (Decoded): [ 1 10  8  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [ ]:
# Example testing on a Latin input
input_seq = input_tensor[0:1]  # Sample Latin input sequence (reshape it to match the input format)
# Decode the sequence to get the Devanagari transliteration
decoded_sentence = decode_sequence(input_seq, encoder_model, decoder_model, targ_tokenizer, max_target_len)

# Print the predicted Devanagari transliteration
print("Predicted Devanagari Transliteration:", decoded_sentence)


NameError: name 'encoder_model' is not defined